# Simple peptide simulations

### Imports

In [1]:
import mdtraj as md
import numpy as np
import scipy
from scipy import ndimage
from scipy import misc
from sklearn import (manifold, datasets, decomposition, ensemble, discriminant_analysis, random_projection)
import reconstruct_video as rv
import networkx as nx
import evaluate_traj as ev
import copy as cp

### AK peptide reconstruction with (x,y,z)

In [2]:
# Read Data and Apply ISOMAP

# read data
t = md.load('.\\Data\\akpeptide\\ak19_2500ps.dcd', top='.\\Data\\akpeptide\\ak19.pdb')
t = t[0:1000]

# Get trajectory info. reshape to prep for isomap
num_frames, num_atoms, num_axis = t.xyz.shape
X = np.reshape(t.xyz, (num_frames, num_atoms*num_axis))

# Shuffle the frames!!!!!!
np.random.shuffle(X)

# apply isomap
n_neighbors = 40
n_components = 25
X_iso = manifold.Isomap(n_neighbors, n_components).fit_transform(X)

In [3]:
# Save shuffled frames as dcd, temp is the shuffled md_obj
shuffled_traj = cp.deepcopy(t)
shuffled_traj.xyz = np.reshape(X, (num_frames, num_atoms, num_axis))
shuffled_traj.save_dcd('raw_xyz_shuffled.dcd')

In [4]:
reload(rv)
# Reconstruct the video using graph algorithms
minG = rv.get_minimum_spanning_tree(X_iso)
order = rv.get_path_from_MSP(minG)

In [5]:
# Get reconstructed, new md_obj
reconstructed_traj = rv.get_reconstructed_traj(shuffled_traj, order)

# Save reconstructed frames as dcd
rv.save_new_trajectory(reconstructed_traj, 'raw_xyz_reconst.dcd')

In [6]:
# Calculate RMSD error between frames
reload(ev)
error = ev.get_RMSD_error(reconstructed_traj)
print 'RMSD error between frames:', error

RMSD error between frames: 3.41424966828


### AK peptide reconstruction with (psi, psi)

In [7]:
# read data
t = md.load('.\\Data\\akpeptide\\ak19_2500ps.dcd', top='.\\Data\\akpeptide\\ak19.pdb')
t = t[0:1000]

# Get trajectory info. get phi shi angle data
phi_idx, phi = md.compute_phi(t)
psi_idx, psi = md.compute_psi(t)
angles = np.concatenate((phi, psi), axis=1)
num_frames, num_angles = angles.shape
#shuffle_idx = np.random.permutation(num_frames)
shuffle_idx = range(num_frames)
angles = angles[shuffle_idx, :]

# apply isomap
n_neighbors = 5
n_components = 10
iso_angles = manifold.Isomap(n_neighbors, n_components).fit_transform(angles)

In [8]:
# Save shuffled frames as dcd, temp is the shuffled md_obj
shuffled_traj = t
shuffled_traj.xyz = shuffled_traj.xyz[shuffle_idx, :, :]
shuffled_traj.save_dcd('phi_psi_shuffled.dcd')

In [9]:
reload(rv)
# Reconstruct the video using graph algorithms
minG = rv.get_minimum_spanning_tree(iso_angles)
order = rv.get_path_from_MSP(minG)

In [31]:
# Get reconstructed, new md_obj
reconstructed_traj = rv.get_reconstructed_traj(shuffled_traj, order)

# Save reconstructed frames as dcd
rv.save_new_trajectory(reconstructed_traj, 'raw_xyz_reconst.dcd')

In [32]:
# Calculate RMSD error between frames
reload(ev)
error = ev.get_RMSD_error(reconstructed_traj)
print 'RMSD error between frames:', error

RMSD error between frames: 7.74666839128


### AK peptide reconstruction with (x, y, z) with alignment

In [2]:
# Read Data and Apply ISOMAP

# read data
t = md.load('.\\Data\\akpeptide\\ak19_2500ps.dcd', top='.\\Data\\akpeptide\\ak19.pdb')
t = t[0:1000]
t.superpose(t)

# Get trajectory info. reshape to prep for isomap
num_frames, num_atoms, num_axis = t.xyz.shape
X = np.reshape(t.xyz, (num_frames, num_atoms*num_axis))
np.random.shuffle(X)

# apply isomap
n_neighbors = 40
n_components = 30
X_iso = manifold.Isomap(n_neighbors, n_components).fit_transform(X)

In [37]:
# Save shuffled frames as dcd, temp is the shuffled md_obj
shuffled_traj = t
shuffled_traj.xyz = np.reshape(X, (num_frames, num_atoms, num_axis))
shuffled_traj.save_dcd('aligned_xyz_shuffled.dcd')

In [38]:
reload(rv)
# Reconstruct the video using graph algorithms
minG = rv.get_minimum_spanning_tree(X_iso)
order = rv.get_path_from_MSP(minG)

In [39]:
# Get reconstructed, new md_obj
reconstructed_traj = rv.get_reconstructed_traj(shuffled_traj, order)

# Save reconstructed frames as dcd
rv.save_new_trajectory(reconstructed_traj, 'aligned_xyz_reconst.dcd')

In [40]:
# Calculate RMSD error between frames
reload(ev)
error = ev.get_RMSD_error(reconstructed_traj)
print 'RMSD error between frames:', error

RMSD error between frames: 4.12488003269


In [3]:
num_atoms

229L